In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_keywords_tfidf(text, top_k=10):
    vectorizer = TfidfVectorizer(
        #stop_words='russian',
        ngram_range=(1, 2),     # учитывать uni- и bi-grams
        max_features=5000
    )

    tfidf_matrix = vectorizer.fit_transform([text])
    scores = tfidf_matrix.toarray()[0]

    indices = np.argsort(scores)[::-1][:top_k]
    features = vectorizer.get_feature_names_out()

    return [features[i] for i in indices]


text = """
Машинное обучение является одним из наиболее важных направлений в области искусственного интеллекта.
Оно позволяет компьютерам самостоятельно извлекать знания из данных, находить закономерности и улучшать свою производительность путем накопления опыта.
Благодаря этому машинное обучение широко применяется в таких областях, как распознавание образов,
обработка естественного языка, рекомендательные системы, финансовое моделирование и многое другое.
"""

print(extract_keywords_tfidf(text, top_k=10))


['обучение', 'машинное', 'машинное обучение', 'из', 'этому машинное', 'языка', 'является одним', 'является', 'языка рекомендательные', 'финансовое']


In [ ]:
!pip install rake-nltk


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from rake_nltk import Rake

def extract_keywords_rake(text, top_k=10):
    rake = Rake()
    rake.extract_keywords_from_text(text)
    ranked_phrases = rake.get_ranked_phrases()
    return ranked_phrases[:top_k]

#text = "Natural language processing enables computers to understand human language."
print(extract_keywords_rake(text))


['машинное обучение является одним из наиболее важных направлений в области искусственного интеллекта', 'благодаря этому машинное обучение широко применяется в таких областях', 'находить закономерности и улучшать свою производительность путем накопления опыта', 'оно позволяет компьютерам самостоятельно извлекать знания из данных', 'финансовое моделирование и многое другое', 'обработка естественного языка', 'как распознавание образов', 'рекомендательные системы']


keybert

In [ ]:
!pip install keybert


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.0 MB/s eta 0:00:00


In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT(model="all-MiniLM-L6-v2")

def extract_keywords_keybert(text, top_k=10):
    keywords = kw_model.extract_keywords(
        text,
        top_n=top_k,
        keyphrase_ngram_range=(1, 2),
        stop_words='english'
    )
    return [kw for kw, score in keywords]

text = "Transformers have revolutionized natural language processing and keyword extraction."
print(extract_keywords_keybert(text))


['keyword extraction', 'transformers revolutionized', 'language processing', 'transformers', 'natural language', 'processing keyword', 'revolutionized natural', 'keyword', 'revolutionized', 'language']


In [ ]:
!pip install sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

model = SentenceTransformer("all-MiniLM-L6-v2")

def extract_keywords_hf(text, top_k=10):
    words = list(set(word_tokenize(text)))
    words = [w for w in words if w.isalpha()]  # только слова

    doc_embedding = model.encode([text], convert_to_tensor=True)
    word_embeddings = model.encode(words, convert_to_tensor=True)

    cosine_scores = util.pytorch_cos_sim(doc_embedding, word_embeddings)[0]
    top_indices = cosine_scores.argsort(descending=True)[:top_k]

    return [words[i] for i in top_indices]

text = "Deep learning allows machines to learn complex patterns from data."
print(extract_keywords_hf(text))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['patterns', 'learning', 'Deep', 'machines', 'data', 'learn', 'complex', 'allows', 'from', 'to']
